In [149]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2 

from tqdm import tqdm

In [148]:
import torch
from torch import nn,optim
from torchvision import transforms
from torch.utils.data import DataLoader, random_split

In [138]:
from models import VGG13

In [116]:
pictures={
    'object_1' : "/home/ychien13/my_github/CSE467_Privacy/Image_Poisoning-/data/raw/Images/objects/object_1.jpg",
    'female_1' : "/home/ychien13/my_github/CSE467_Privacy/Image_Poisoning-/data/raw/Images/female_faces/female_2.jpg",
    'male_1' : "/home/ychien13/my_github/CSE467_Privacy/Image_Poisoning-/data/raw/Images/male_faces/male_3.jpg"
}

In [118]:
for keys in pictures.keys():
    img = cv2.imread(pictures[keys])
    print(img.shape)

(627, 940, 3)
(650, 531, 3)
(627, 940, 3)


In [119]:
class FaceDataset():
    
    def __init__(self, root_path:str, transform=None):
        self.root_path=root_path
        
        self.transform = transform

        self.img_labels={dir_name:i for i,dir_name in enumerate(sorted(os.listdir(root_path)))}
        
        self.samples:list[tuple(str,int)] = []

        for folder_name in os.listdir(self.root_path):
            
            folder_path:str = os.path.join(self.root_path,folder_name)
            
            for img in os.listdir(folder_path):
                
                try:
                    self.samples.append(( os.path.join(folder_path,img), self.img_labels[folder_name]))
                except Exception as e:
                    print(f'{e}')

    def __len__(self):
        return len(self.samples)
        
    def __getitem__(self,index):
        
        img_path, label = self.samples[index]
        img = cv2.imread(img_path)
        if transform:
            img = self.transform(img)

        return (img, label)
        

In [134]:
def get_mean_std():
    ...

In [232]:
# Training Variable

ROOT_PATH='../data/raw/Images'
IN_CHANNELS=3
OUT_CHANNELS=3
BATCH_SIZE = 8
LEARNING_RATE=0.01
EPOCHS=1000

# Check is nvidia GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224),antialias=False)
])

# Initializing Datasets
dataset = FaceDataset(ROOT_PATH,transform)
train_data,attack_data, test_data = random_split(dataset,[0.7,0.1,0.2])
print('Total number of dataset:',len(dataset))
print('Train                  :',len(train_data))
print('Attack                 :', len(attack_data))
print('Test                   :', len(test_data))

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

# Setting up 
model = VGG13(IN_CHANNELS,OUT_CHANNELS).to(device)

optimizer = optim.AdamW( model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

Total number of dataset: 5000
Train                  : 3500
Attack                 : 500
Test                   : 1000


In [ ]:
# data, targets = next(iter(train_loader))

for epoch in range(EPOCHS):
    model.train()
    correct=0
    # for idx, (data, targets) in enumerate(train_loader):
        
    data = data.to(device)
    targets = targets.to(device)
    # Forward
    prediction = model(data)
    
    loss = criterion(prediction,targets)

    # Backward
    optimizer.zero_grad()
    loss.backward()
    # print(loss)
    optimizer.step()
    
    # Calculate numer of correct classification
    correct+=(prediction.argmax(1) == targets).sum().item()
    
    print(f'Epoch [{epoch+1}/{EPOCHS}] | {correct}/{len(train_data)} | Accuracy: {correct/len(train_data):.2f}')
    

Epoch [1/1000] | 1/3500 | Accuracy: 0.00
Epoch [2/1000] | 4/3500 | Accuracy: 0.00
Epoch [3/1000] | 3/3500 | Accuracy: 0.00
Epoch [4/1000] | 3/3500 | Accuracy: 0.00
Epoch [5/1000] | 3/3500 | Accuracy: 0.00
Epoch [6/1000] | 1/3500 | Accuracy: 0.00
Epoch [7/1000] | 4/3500 | Accuracy: 0.00
Epoch [8/1000] | 3/3500 | Accuracy: 0.00
Epoch [9/1000] | 3/3500 | Accuracy: 0.00
Epoch [10/1000] | 3/3500 | Accuracy: 0.00
Epoch [11/1000] | 3/3500 | Accuracy: 0.00
Epoch [12/1000] | 1/3500 | Accuracy: 0.00
Epoch [13/1000] | 1/3500 | Accuracy: 0.00
Epoch [14/1000] | 4/3500 | Accuracy: 0.00
Epoch [15/1000] | 4/3500 | Accuracy: 0.00
Epoch [16/1000] | 4/3500 | Accuracy: 0.00
Epoch [17/1000] | 4/3500 | Accuracy: 0.00
Epoch [18/1000] | 3/3500 | Accuracy: 0.00
Epoch [19/1000] | 4/3500 | Accuracy: 0.00
Epoch [20/1000] | 3/3500 | Accuracy: 0.00
Epoch [21/1000] | 3/3500 | Accuracy: 0.00
Epoch [22/1000] | 3/3500 | Accuracy: 0.00
Epoch [23/1000] | 3/3500 | Accuracy: 0.00
Epoch [24/1000] | 1/3500 | Accuracy: 0.00
E

In [227]:
data, targets = next(iter(train_loader))

print(targets.dtype,targets.shape)

torch.int64 torch.Size([8])
